In [231]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch 

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms

from PIL import Image

from math import floor

import torch.nn.functional as F

from sklearn.preprocessing import MultiLabelBinarizer

import os

import torchvision.models as models

%matplotlib inline

In [113]:
csv_data = pd.read_csv("train_v2.csv")
#data = np.array(np.zeros((500,256,256,3)),dtype=np.float32)

In [114]:
#def DataLoader(data,batch_size = 10):
#    i = 0
#    ret_data = []
#    while True:
#        
#        if i+batch_size > images_names.shape[0]:
#            get_data = images_names[i:]
#            i=0
#        else:
#            get_data = images_names[i:i+batch_size]
#            i += batch_size
#            
#        for image in get_data:
#            ret_data.append(plt.imread("train-jpg/%s.jpg" % image)[:,:,0:3].reshape(3,256,256))#
#
#        
#        yield torch.from_numpy(np.array(ret_data).astype('float32')).type(torch.FloatTensor)

In [115]:
def train_valid_split(dataframe, test_size = 0.25, shuffle = False, random_seed = 0):
    """ Return a list of splitted indices from a DataSet.
    Indices can be used with DataLoader to build a train and validation set.
    
    Arguments:
        A Dataframe
        A test_size, as a float between 0 and 1 (percentage split) or as an int (fixed number split)
        Shuffling True or False
        Random seed
    """
    length = len(dataframe)
    indices = list(range(1,length))
    
    if shuffle == True:
        random.seed(random_seed)
        random.shuffle(indices)
    
    if type(test_size) is float:
        split = floor(test_size * length)
    elif type(test_size) is int:
        split = test_size
    else:
        raise ValueError('%s should be an int or a float' % str)
    return indices[split:], indices[:split]

In [116]:
class KaggleAmazonDataset(Dataset):
    """Dataset wrapping images and target labels for Kaggle - Planet Amazon from Space competition.

    Arguments:
        A CSV file path
        Path to image folder
        Extension of images
        PIL transforms
    """

    def __init__(self, csv_path, img_path, img_ext, transform=None):
    
        tmp_df = pd.read_csv(csv_path)
        assert tmp_df['image_name'].apply(lambda x: os.path.isfile(img_path + x + img_ext)).all(), \
"Some images referenced in the CSV file were not found"
        
        self.mlb = MultiLabelBinarizer()
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform

        self.X_train = tmp_df['image_name']
        self.y_train = self.mlb.fit_transform(tmp_df['tags'].str.split()).astype(np.float32)

    def __getitem__(self, index):
        img = Image.open(self.img_path + self.X_train[index] + self.img_ext)
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        label = torch.from_numpy(self.y_train[index])
        return img, label

    def __len__(self):
        return len(self.X_train.index)

In [220]:
#data = DataLoader(csv_data["image_name"])
#label = DataLoader(csv_data["tags"])
#for i,data in enumerate(data):
    #print(i),print(data)
IMG_PATH = 'train-jpg/'
IMG_EXT = '.jpg'
TRAIN_DATA = 'train_v2.csv'
#transformations = transforms.Compose([transforms.Scale(32),transforms.ToTensor()])
#dset_train = KaggleAmazonDataset(TRAIN_DATA,IMG_PATH,IMG_EXT,transformations)


## Augmentation + Normalization for full training
ds_transform_augmented = transforms.Compose([
                 #transforms.RandomSizedCrop(224),
                 transforms.Scale(512),
                 transforms.RandomHorizontalFlip(),
                 #transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                # std=[0.229, 0.224, 0.225]),
                 transforms.ToTensor(),
                 #

                 # Affine(
                 #     rotation_range = 15,
                 #     translation_range = (0.2,0.2),
                 #     shear_range = math.pi/6,
                 #     zoom_range=(0.7,1.4)
                 # )
])

## Normalization only for validation and test
ds_transform_raw = transforms.Compose([
                 transforms.Scale(224),
                 transforms.ToTensor(),
                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
                 ])

####     #########     ########     ###########     #####

X_train = KaggleAmazonDataset(TRAIN_DATA,IMG_PATH,IMG_EXT,
                             ds_transform_augmented
                             )
X_val = KaggleAmazonDataset(TRAIN_DATA,IMG_PATH,IMG_EXT,
                             ds_transform_raw
                             )

In [221]:
# Creating a validation split
train_idx, valid_idx = train_valid_split(X_train, 0.2)

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(X_train,
                          batch_size=2,
                          sampler=train_sampler,
                          num_workers=4,
                         )

valid_loader = DataLoader(X_val,
                          batch_size=2,
                          sampler=valid_sampler,
                          num_workers=4)

In [222]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(186624, 256)
        self.fc2 = nn.Linear(256, 17)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1) # Flatten layer
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)
    
    def train_generator(model,epoch,train_loader,optimizer,loss):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
            data, target = Variable(data), Variable(target)
            optimizer.zero_grad()
            output = model(data)
            loss = F.binary_cross_entropy(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))
                
    def test_generator(model, val_loader, loss):
        model.eval()
        test_loss = 0
        ammount = 0
        for data, target in val_loader:
            data, target = Variable(data.float(), volatile=True), Variable(target.float())
            output = model(data)
            ammount += len(data)
            test_loss += loss(output, target).data[0]

        test_loss /= ammount
        print('Test set: Average loss: %s\n' % test_loss)

In [ ]:
model = Net()
model = models.inception_v3(num_classes=17, aux_logits=False)
optimizer = optim.RMSprop(model.parameters(),lr=0.00001)
optimizer = optim.Adam(model.parameters(),lr=0.0001)

In [234]:
def train_generator(model,epoch,train_loader,optimizer,loss):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
            data, target = Variable(data), Variable(target)
            optimizer.zero_grad()
            output = model(data)
            loss = F.binary_cross_entropy(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))
                
def test_generator(model, val_loader, loss):
        model.eval()
        test_loss = 0
        ammount = 0
        for data, target in val_loader:
            data, target = Variable(data.float(), volatile=True), Variable(target.float())
            output = model(data)
            ammount += len(data)
            test_loss += loss(output, target).data[0]

        test_loss /= ammount
        print('Test set: Average loss: %s\n' % test_loss)

In [235]:
for epoch in range(1, 1 + 1):
    train_generator(model,epoch,train_loader,optimizer,MSELoss)
    #test_generator(model,valid_loader,MSELoss)

Train Epoch: 1 [0/40479 (0%)]	Loss: nan


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/diego/.conda/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-235-9c9534378e9a>", line 2, in <module>
    train_generator(model,epoch,train_loader,optimizer,MSELoss)
  File "<ipython-input-234-2571e44fe250>", line 7, in train_generator
    output = model(data)
  File "/home/diego/.conda/envs/tensorflow/lib/python3.6/site-packages/torch/nn/modules/module.py", line 206, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/diego/.conda/envs/tensorflow/lib/python3.6/site-packages/torchvision/models/inception.py", line 114, in forward
    x = self.Mixed_7c(x)
  File "/home/diego/.conda/envs/tensorflow/lib/python3.6/site-packages/torch/nn/modules/module.py", line 206, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/diego/.conda/envs/tensorflow/lib/python3.6/site-packages/torchv

KeyboardInterrupt: 